In [1]:
import pandas as pd
import numpy as np

In [28]:
loc_names = ['time', 'ign1', 'ign2', 'accuracy', 'latitude', 'longitude', 'altitude']
loc = pd.read_table('data/train/Location.txt', header = None, names = tmp_names, sep = " ")

In [37]:
gps_names = ['time', 'ign1', 'ign2', 'id', 'snr', 'azimuth', 'elevation']
gps = pd.read_table('data/train/GPS.txt', header = None, names = gps_names, sep = " ")

In [39]:
gps

,,,,,time,ign1,ign2,id,snr,azimuth,elevation
1490430924008,54415677594,242658854,7,18.0,62.0,50.0,9.0,17.0,86.0,24.0,2.0
1490430925005,55412323428,242658854,7,18.0,62.0,50.0,9.0,17.0,86.0,24.0,2.0
1490430926011,56418411969,242658854,7,18.0,62.0,50.0,9.0,18.0,86.0,24.0,2.0
1490430927018,57425188011,242658854,7,18.0,62.0,50.0,9.0,19.0,86.0,24.0,2.0
1490430928004,58410924469,242658854,7,17.0,62.0,50.0,9.0,19.0,86.0,24.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
1499268456045,25743270261693,149674581225,18,14.0,75.0,45.0,67.0,18.0,76.0,42.0,82.0
1499268457048,25744273048672,149674581225,18,13.0,75.0,45.0,67.0,18.0,76.0,42.0,82.0
1499268458049,25745274413256,149674581225,18,13.0,75.0,45.0,67.0,18.0,76.0,42.0,82.0
1499268459048,25746273234610,149674581225,18,13.0,75.0,45.0,67.0,18.0,76.0,42.0,82.0
